# Comparaison des modèles - Credit Scoring
    Ce notebook entraîne plusieurs modèles et compare leurs performances.
- Modèles : Logistic Regression, XGBoost, Neural Network (TensorFlow/Keras)
- Métriques : AUC, F1-score
- Tracking : MLflow

## Import des bibliothèques

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import mlflow
import mlflow.sklearn
import mlflow.tensorflow

In [ ]:
# Charger les données
df = pd.read_csv('../data/processed/processed_data.csv')
X = df.drop('target', axis=1)  # remplacer 'target' par la colonne cible
y = df['target']

# Split train/validation/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scaling des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## Fonction pour évaluer et logger les modèles

In [ ]:
# Fonction pour évaluer et logger les modèles
def evaluate_model(model, X_val, y_val, X_test, y_test, model_name):
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)

    print("{model_name} - Test AUC:" {auc:.4f}, F1: {f1:.4f})

    # MLflow logging
    with mlflow.start_run(run_name=model_name):
    mlflow.log_param('model', model_name)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.sklearn.log_model(model, model_name)
    return auc, f1

## 1️⃣ Logistic Regression"

In [ ]:
lr_model = LogisticRegression(class_weight='balanced', max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
evaluate_model(lr_model, X_val_scaled, y_val, X_test_scaled, y_test, 'LogisticRegression')

## 2️⃣ XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=sum(y_train==0)/sum(y_train==1))
xgb_model.fit(X_train, y_train)
evaluate_model(xgb_model, X_val, y_val, X_test, y_test, 'XGBoost')

## 3️⃣ Neural Network (TensorFlow / Keras)

In [ ]:
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
    
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'F1Score'])

nn_model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=20, batch_size=32)

# MLflow tracking TensorFlow
with mlflow.start_run(run_name='NeuralNetwork'):
    mlflow.tensorflow.log_model(nn_model, 'NeuralNetwork')
    y_test_pred_nn = (nn_model.predict(X_test_scaled) > 0.5).astype(int)
    auc = roc_auc_score(y_test, y_test_pred_nn)
    f1 = f1_score(y_test, y_test_pred_nn)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    print("NeuralNetwork - Test AUC:" {auc:.4f}, F1: {f1:.4f})

## ✅ Conclusion

- Comparer les métriques AUC et F1 pour sélectionner le meilleur modèle.
- Le modèle retenu sera exporté pour industrialisation (User Story 3.3).